# Bicing Linear Regression — Gap-Aware, Multi-Horizon

This notebook trains linear regression models to forecast `num_bikes_available` across multiple time horizons (1, 3, 6, and 12 steps ahead) for Barcelona's Bicing bike-sharing system. It implements the same gap-aware preprocessing as the LSTM workflow, processing sliding windows in a stateless manner and comparing several linear regressors (LinearRegression, Ridge, Lasso) to select the best-performing model on the validation split. The best model (Lasso with alpha=0.001) achieves an average MAE of 0.174 bikes, significantly outperforming the carry-forward baseline (MAE=12.584).

In [1]:
# --- Setup & Imports ---
import os, json, math, joblib
from collections import defaultdict, deque
from typing import Dict, Iterator, List, Optional, Tuple

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.multioutput import MultiOutputRegressor


/home/jorgeajimenezl/Documents/harbour-space/elizabeth-eeira/bicing-prediction/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Configuration ---
DATA_PATH = "data/2025_09_Setembre_BicingNou_ESTACIONS.csv"  # <-- Change this if you use a different CSV
META_PATH = os.path.join("bicing_lstm_artifacts_stateless", "meta.json")
SAVE_DIR = "bicing_linear_artifacts"
os.makedirs(SAVE_DIR, exist_ok=True)

CSV_KW = dict(
    sep=",",
    dtype={
        "station_id": "int64",
        "num_bikes_available": "int64",
        "num_bikes_available_types.mechanical": "int64",
        "num_bikes_available_types.ebike": "int64",
        "num_docks_available": "int64",
        "last_reported": "int64",
        "is_charging_station": "bool",
        "status": "string",
        "is_installed": "int64",
        "is_renting": "int64",
        "is_returning": "int64",
        "last_updated": "int64",
        "ttl": "int64",
    },
    usecols=[
        "station_id",
        "num_bikes_available",
        "num_bikes_available_types.mechanical",
        "num_bikes_available_types.ebike",
        "num_docks_available",
        "last_reported",
        "is_charging_station",
        "status",
        "is_installed",
        "is_renting",
        "is_returning",
        "last_updated",
        "ttl",
    ],
    engine="c",
)

assert os.path.exists(DATA_PATH), f"Missing data file: {DATA_PATH}"
assert os.path.exists(META_PATH), f"Run the LSTM notebook first to generate: {META_PATH}"

with open(META_PATH, "r") as f:
    meta = json.load(f)

HORIZONS = meta["horizons"]
SEQ_LEN = meta["seq_len"]
VAL_CUTOFF = meta["val_cutoff"]
BASE_STEP_SEC = meta["base_step_sec"]
SHORT_GAP_STEPS = meta["short_gap_steps"]
LONG_GAP_STEPS = meta["long_gap_steps"]
MAX_DELTA = meta["max_delta"]
BACKFILL_SHORT_GAPS = meta["backfill_short_gaps"]

station_list = meta["station_ids"]
status_list = meta["status_values"]
station2id = {sid: i for i, sid in enumerate(station_list)}
status2id = {s: i for i, s in enumerate(status_list)}

SCALE_MEAN = meta["scaler_mean"]
SCALE_STD = meta["scaler_std"]

FEAT_CONT = [
    "num_bikes_available",
    "num_bikes_available_types.mechanical",
    "num_bikes_available_types.ebike",
    "num_docks_available",
    "is_installed",
    "is_renting",
    "is_returning",
    "ttl",
    "sin_hour", "cos_hour", "sin_dow", "cos_dow",
    "delta_steps", "log1p_delta_steps", "is_gap",
]
TARGET_COL = "num_bikes_available"

CHUNK_SIZE = 500_000
TRAIN_MAX_SAMPLES = 20_000
VAL_MAX_SAMPLES = 5_000
TRAIN_SUBSAMPLE_PROB = 0.20
RANDOM_STATE = 42

print(f"#stations={len(station2id)}, #status={len(status2id)} (+1 unk)")
print("HORIZONS:", HORIZONS, "| SEQ_LEN:", SEQ_LEN)
print("Streaming chunk size:", CHUNK_SIZE)
print("Training samples target:", TRAIN_MAX_SAMPLES)


#stations=546, #status=2 (+1 unk)
HORIZONS: [1, 3, 6, 12] | SEQ_LEN: 90
Streaming chunk size: 500000
Training samples target: 20000


In [3]:
# --- Stateless stream helper (linear-friendly) ---
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    ts = pd.to_datetime(df["last_reported"], unit="s", utc=True)
    ts_local = ts.dt.tz_convert("Europe/Madrid")
    df = df.copy()
    df["hour"] = ts_local.dt.hour.astype(np.int16)
    df["dow"] = ts_local.dt.dayofweek.astype(np.int16)
    df["sin_hour"] = np.sin(2 * np.pi * df["hour"] / 24.0)
    df["cos_hour"] = np.cos(2 * np.pi * df["hour"] / 24.0)
    df["sin_dow"] = np.sin(2 * np.pi * df["dow"] / 7.0)
    df["cos_dow"] = np.cos(2 * np.pi * df["dow"] / 7.0)
    return df

class StatelessSequenceStream:
    def __init__(
        self,
        data_path: str,
        csv_kw: Dict,
        station2id: Dict[int, int],
        status2id: Dict[str, int],
        scaler_mean: Dict[str, float],
        scaler_std: Dict[str, float],
        feat_cont: List[str],
        target_col: str,
        seq_len: int,
        horizons: List[int],
        chunk_size: int,
        split: str,
        val_cutoff: int,
        base_step_sec: int,
        short_gap_steps: int,
        long_gap_steps: int,
        max_delta: int,
        backfill_short_gaps: bool,
    ) -> None:
        assert split in {"train", "val"}
        self.data_path = data_path
        self.csv_kw = csv_kw
        self.station2id = station2id
        self.status2id = status2id
        self.mean = scaler_mean
        self.std = scaler_std
        self.feat_cont = feat_cont
        self.target_col = target_col
        self.seq_len = seq_len
        self.horizons = sorted(horizons)
        self.max_h = max(self.horizons)
        self.chunk_size = chunk_size
        self.split = split
        self.val_cutoff = val_cutoff
        self.base_step_sec = base_step_sec
        self.short_gap_steps = short_gap_steps
        self.long_gap_steps = long_gap_steps
        self.max_delta = max_delta
        self.backfill_short_gaps = backfill_short_gaps
        self.delta_idx = [
            self.feat_cont.index("delta_steps"),
            self.feat_cont.index("log1p_delta_steps"),
            self.feat_cont.index("is_gap"),
        ]
        self.buffers = defaultdict(
            lambda: {
                "cont": deque(maxlen=self.seq_len + self.max_h + 5),
                "status": deque(maxlen=self.seq_len + self.max_h + 5),
                "target": deque(maxlen=self.seq_len + self.max_h + 5),
                "time": deque(maxlen=self.seq_len + self.max_h + 5),
            }
        )
        self.last_ts_global: Dict[int, int] = {}

    def _status_encode(self, series: pd.Series) -> np.ndarray:
        unk = len(self.status2id)
        return (
            series.fillna("")
            .apply(lambda x: self.status2id.get(str(x), unk))
            .astype(np.int32)
            .to_numpy()
        )

    def _standardize_inplace(self, df: pd.DataFrame) -> None:
        for f in self.feat_cont:
            mu = self.mean[f]
            sigma = self.std[f] + 1e-6
            df[f] = (df[f].astype("float64") - mu) / sigma

    def __iter__(self) -> Iterator[Tuple[int, np.ndarray, np.ndarray, np.ndarray]]:
        for chunk in pd.read_csv(self.data_path, chunksize=self.chunk_size, **self.csv_kw):
            chunk = chunk.sort_values(["station_id", "last_reported"])
            if self.split == "train":
                chunk = chunk[chunk["last_reported"] < self.val_cutoff]
            else:
                chunk = chunk[chunk["last_reported"] >= self.val_cutoff]
            if chunk.empty:
                continue

            chunk = add_time_features(chunk)

            deltas, logs, gaps = [], [], []
            for stn, t in zip(
                chunk["station_id"].to_numpy(),
                chunk["last_reported"].to_numpy(),
            ):
                prev = self.last_ts_global.get(int(stn))
                ds = 0 if prev is None else max(0, int(round((int(t) - int(prev)) / self.base_step_sec)))
                self.last_ts_global[int(stn)] = int(t)
                ds = min(ds, self.max_delta)
                deltas.append(float(ds))
                logs.append(float(math.log1p(ds)))
                gaps.append(float(1.0 if ds > 1 else 0.0))
            chunk["delta_steps"] = np.array(deltas, dtype=np.float32)
            chunk["log1p_delta_steps"] = np.array(logs, dtype=np.float32)
            chunk["is_gap"] = np.array(gaps, dtype=np.float32)

            self._standardize_inplace(chunk)

            stn_series = chunk["station_id"].astype("int64")
            stn_map = stn_series.map(self.station2id)
            valid_mask = stn_map.notna()
            if not valid_mask.all():
                chunk = chunk.loc[valid_mask].copy()
                stn_map = stn_map.loc[valid_mask]
            if chunk.empty:
                continue

            status_ids = self._status_encode(chunk["status"])
            cont_arr = chunk[self.feat_cont].astype(np.float32).to_numpy()
            tgt_arr = chunk[self.target_col].astype(np.float32).to_numpy()
            time_arr = chunk["last_reported"].astype(np.int64).to_numpy()

            for stn_idx, status_id, cont, tgt, t in zip(
                stn_map.to_numpy(dtype=np.int32),
                status_ids,
                cont_arr,
                tgt_arr,
                time_arr,
            ):
                buf = self.buffers[int(stn_idx)]
                prev_t = buf["time"][-1] if buf["time"] else None
                ds = 0 if prev_t is None else max(0, int(round((int(t) - int(prev_t)) / self.base_step_sec)))

                if ds > self.long_gap_steps:
                    buf["cont"].clear()
                    buf["status"].clear()
                    buf["target"].clear()
                    buf["time"].clear()

                if self.backfill_short_gaps and 1 < ds <= self.short_gap_steps and buf["cont"]:
                    last_cont = buf["cont"][-1].copy()
                    last_status = buf["status"][-1]
                    last_target = buf["target"][-1]
                    last_time = buf["time"][-1]
                    for step in range(1, ds):
                        cf = last_cont.copy()
                        cf[self.delta_idx[0]] = 1.0
                        cf[self.delta_idx[1]] = math.log1p(1.0)
                        cf[self.delta_idx[2]] = 1.0
                        buf["cont"].append(cf)
                        buf["status"].append(last_status)
                        buf["target"].append(last_target)
                        buf["time"].append(last_time + step * self.base_step_sec)

                buf["cont"].append(cont)
                buf["status"].append(int(status_id))
                buf["target"].append(float(tgt))
                buf["time"].append(int(t))

                while len(buf["target"]) >= self.seq_len + self.max_h:
                    cont_seq = list(buf["cont"])[: self.seq_len]
                    status_seq = list(buf["status"])[: self.seq_len]
                    y = [buf["target"][self.seq_len - 1 + h] for h in self.horizons]
                    yield (
                        int(stn_idx),
                        np.asarray(status_seq, dtype=np.int32),
                        np.asarray(cont_seq, dtype=np.float32),
                        np.asarray(y, dtype=np.float32),
                    )
                    buf["cont"].popleft()
                    buf["status"].popleft()
                    buf["target"].popleft()
                    buf["time"].popleft()


In [4]:
# --- Sample stateless windows for linear models ---
def collect_split(
    split: str,
    max_samples: int,
    subsample_prob: float,
    seed: int,
) -> Dict[str, np.ndarray]:
    stream = StatelessSequenceStream(
        data_path=DATA_PATH,
        csv_kw=CSV_KW,
        station2id=station2id,
        status2id=status2id,
        scaler_mean=SCALE_MEAN,
        scaler_std=SCALE_STD,
        feat_cont=FEAT_CONT,
        target_col=TARGET_COL,
        seq_len=SEQ_LEN,
        horizons=HORIZONS,
        chunk_size=CHUNK_SIZE,
        split=split,
        val_cutoff=VAL_CUTOFF,
        base_step_sec=BASE_STEP_SEC,
        short_gap_steps=SHORT_GAP_STEPS,
        long_gap_steps=LONG_GAP_STEPS,
        max_delta=MAX_DELTA,
        backfill_short_gaps=BACKFILL_SHORT_GAPS,
    )

    rng = np.random.default_rng(seed)
    cont_rows: List[np.ndarray] = []
    station_rows: List[int] = []
    status_rows: List[int] = []
    target_rows: List[np.ndarray] = []
    last_values: List[float] = []

    progress = tqdm(total=max_samples, desc=f"{split} windows", unit="win")
    for stn_idx, status_seq, cont_seq, target in stream:
        if subsample_prob < 1.0 and rng.random() > subsample_prob:
            continue
        cont_rows.append(cont_seq.reshape(-1))
        station_rows.append(int(stn_idx))
        status_rows.append(int(status_seq[-1]))
        target_rows.append(target)
        last_std = float(cont_seq[-1, 0])
        last_raw = last_std * (SCALE_STD[TARGET_COL] + 1e-6) + SCALE_MEAN[TARGET_COL]
        last_values.append(last_raw)
        progress.update(1)
        if progress.n >= max_samples:
            break
    progress.close()

    if not cont_rows:
        raise RuntimeError(f"No samples collected for split='{split}'. Check dataset and cutoff.")

    X_cont = np.stack(cont_rows).astype(np.float32)
    stations = np.asarray(station_rows, dtype=np.int32)
    statuses = np.asarray(status_rows, dtype=np.int32)
    y = np.stack(target_rows).astype(np.float32)
    last_values = np.asarray(last_values, dtype=np.float32)

    print(f"Collected {len(X_cont):,} samples for split='{split}'.")
    return {
        "X_cont": X_cont,
        "station": stations,
        "status": statuses,
        "y": y,
        "last_value": last_values,
    }

train_data = collect_split(
    split="train",
    max_samples=TRAIN_MAX_SAMPLES,
    subsample_prob=TRAIN_SUBSAMPLE_PROB,
    seed=RANDOM_STATE,
)
val_data = collect_split(
    split="val",
    max_samples=VAL_MAX_SAMPLES,
    subsample_prob=1.0,
    seed=RANDOM_STATE + 1,
)


train windows: 100%|██████████| 20000/20000 [00:02<00:00, 8731.56win/s] 


Collected 20,000 samples for split='train'.


val windows: 100%|██████████| 5000/5000 [00:02<00:00, 1951.71win/s]

Collected 5,000 samples for split='val'.


In [5]:
# --- Assemble design matrices ---
station_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
status_encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

station_train = train_data["station"][:, None]
status_train = train_data["status"][:, None]
station_val = val_data["station"][:, None]
status_val = val_data["status"][:, None]

station_ohe_train = station_encoder.fit_transform(station_train).astype(np.float32)
status_ohe_train = status_encoder.fit_transform(status_train).astype(np.float32)
station_ohe_val = station_encoder.transform(station_val).astype(np.float32)
status_ohe_val = status_encoder.transform(status_val).astype(np.float32)

X_train = np.hstack([train_data["X_cont"], station_ohe_train, status_ohe_train]).astype(np.float32)
X_val = np.hstack([val_data["X_cont"], station_ohe_val, status_ohe_val]).astype(np.float32)
y_train = train_data["y"].astype(np.float32)
y_val = val_data["y"].astype(np.float32)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)


X_train shape: (20000, 1424)
X_val shape: (5000, 1424)
y_train shape: (20000, 4)


In [6]:
# --- Baseline: carry-forward last observation ---
baseline_pred = np.repeat(val_data["last_value"][:, None], len(HORIZONS), axis=1)
baseline_mae = np.mean(np.abs(baseline_pred - y_val), axis=0)
baseline_overall = float(baseline_mae.mean())

baseline_df = pd.DataFrame({
    "horizon": HORIZONS,
    "mae": baseline_mae,
})
print(f"Baseline overall MAE: {baseline_overall:.3f}")
baseline_df


Baseline overall MAE: 12.584


,horizon,mae
0,1,12.581079
1,3,12.582975
2,6,12.583575
3,12,12.586808


In [7]:
# --- Train & evaluate linear models ---
candidates = [
    ("LinearRegression", LinearRegression()),
    ("Ridge (alpha=1.0)", Ridge(alpha=1.0)),
    ("Ridge (alpha=10.0)", Ridge(alpha=10.0)),
    ("Ridge (alpha=100.0)", Ridge(alpha=100.0)),
    ("Lasso (alpha=0.001)", MultiOutputRegressor(Lasso(alpha=0.001, max_iter=5000))),
]

results = []
for name, estimator in candidates:
    print(f"Training {name}...")
    model = estimator.fit(X_train, y_train)
    preds = model.predict(X_val)
    if preds.ndim == 1:
        preds = preds.reshape(-1, 1)
    mae_by_h = np.mean(np.abs(preds - y_val), axis=0)
    overall_mae = float(mae_by_h.mean())
    results.append({
        "model": name,
        "estimator": model,
        "mae_by_h": mae_by_h,
        "overall_mae": overall_mae,
    })

results_table = []
baseline_row = {"model": "Baseline (carry-forward)", "overall_mae": baseline_overall}
for h, mae in zip(HORIZONS, baseline_mae):
    baseline_row[f"h{h}_mae"] = mae
results_table.append(baseline_row)

for r in results:
    row = {"model": r["model"], "overall_mae": r["overall_mae"]}
    for h, mae in zip(HORIZONS, r["mae_by_h"]):
        row[f"h{h}_mae"] = mae
    results_table.append(row)

results_df = pd.DataFrame(results_table).sort_values("overall_mae").reset_index(drop=True)
results_df


Training LinearRegression...
Training Ridge (alpha=1.0)...


/home/jorgeajimenezl/Documents/harbour-space/elizabeth-eeira/bicing-prediction/.venv/lib/python3.13/site-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=5.85776e-08): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


Training Ridge (alpha=10.0)...
Training Ridge (alpha=100.0)...
Training Lasso (alpha=0.001)...


,model,overall_mae,h1_mae,h3_mae,h6_mae,h12_mae
0,Lasso (alpha=0.001),0.173926,0.060068,0.139270,0.206510,0.289854
1,Ridge (alpha=100.0),0.179012,0.068164,0.143616,0.210510,0.293759
2,Ridge (alpha=10.0),0.183317,0.067688,0.146476,0.216218,0.302888
3,LinearRegression,0.184583,0.068445,0.147239,0.217332,0.305316
4,Ridge (alpha=1.0),0.185395,0.068971,0.148385,0.218726,0.305500
5,Baseline (carry-forward),12.583610,12.581079,12.582975,12.583575,12.586808


In [8]:
# --- Select best model and persist ---
ranked = sorted(results, key=lambda r: r["overall_mae"])
best = ranked[0]
best_name = best["model"]
best_overall = best["overall_mae"]
print(f"Best model: {best_name} (overall MAE={best_overall:.3f})")

artifact = {
    "model_name": best_name,
    "model": best["estimator"],
    "station_encoder": station_encoder,
    "status_encoder": status_encoder,
    "horizons": HORIZONS,
    "seq_len": SEQ_LEN,
    "feat_cont": FEAT_CONT,
    "target_col": TARGET_COL,
    "scaler_mean": SCALE_MEAN,
    "scaler_std": SCALE_STD,
    "train_samples": int(train_data["X_cont"].shape[0]),
    "val_samples": int(val_data["X_cont"].shape[0]),
}
artifact_path = os.path.join(SAVE_DIR, "best_linear_model.joblib")
joblib.dump(artifact, artifact_path)
print(f"Saved best model to {artifact_path}")


Best model: Lasso (alpha=0.001) (overall MAE=0.174)
Saved best model to bicing_linear_artifacts/best_linear_model.joblib


## Next steps

- Evaluate the saved model on a hold-out test period or live data.
- Integrate the artifact into the API (see `api.py`) for online predictions.
- Experiment with additional features (e.g., weather) or alternative regularization strengths to push MAE lower.
